# Stability Constraints
$$ b_{0j} \le 1 - \gamma $$
$$ b_{1j} - b_{0j} \le 1 - \gamma $$
$$ - b_{1j} \le 1 - \gamma $$

With 

$$ \gamma = 1 - (1 - \epsilon_s)^2 $$


$$ \mathbf{B}\delta \lt \mathbf{b}^{(k)} $$

In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt


plt.rcParams['figure.figsize'] = (20, 8)
plt.rcParams['axes.grid'] = True
                                  
                                  
from dsp import simplify_biquad_filters
import dsp

In [2]:
sos = signal.iirfilter(4, 0.2, rs=60, btype='lowpass',
                       analog=False, ftype='cheby2',
                       output='sos')
c = simplify_biquad_filters(sos)

In [3]:
c

array([ 1.00000000e+00, -3.24370246e-01,  6.95736399e-01, -1.66300199e+00,
        1.00000000e+00, -1.55971254e+00,  8.69492908e-01, -1.83559567e+00,
        1.50402320e-03])

In [6]:
len(c) // 4

2

In [7]:
r_max = 0.98

In [8]:
epsilon_s = 1 - r_max
b = []
J = len(c) // 4 # number of filter stages
for i in range(0, J*4, 4):
    a0, a1, b0, b1 = c[i:i+4]
    b_ = np.c_[(1-epsilon_s)-b0, 
            (1-epsilon_s)-b1+b0, 
            (1-epsilon_s)+b1+b0].T
    b.append(b_)
b = np.vstack(b)

width = 2*J + 2
height = 3*J
B = np.zeros((height,width))
# B = np.zeros((J*3,J*5))
beta = np.array([[1, 0], [-1, 1], [-1, -1]])

for i in range(J):
    x = 2*i
    y = i*3
    B[y:y+3, x:x+2] = beta

In [9]:
b

array([[0.2842636 ],
       [3.33873839],
       [0.01273441],
       [0.11050709],
       [3.68508857],
       [0.01389724]])

In [10]:
B

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.],
       [ 0.,  0., -1., -1.,  0.,  0.]])

In [11]:
np.matmul(B, b)

array([[ 0.2842636 ],
       [ 3.05447479],
       [-3.62300199],
       [ 0.01273441],
       [ 0.09777268],
       [-0.12324151]])

In [12]:
beta = np.array([[1, 0], [-1, 1], [-1, -1]])

In [13]:
beta

array([[ 1,  0],
       [-1,  1],
       [-1, -1]])

In [14]:
B

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.],
       [ 0.,  0., -1., -1.,  0.,  0.]])

# Passband Constraints

The desired magnitude response in the passband = 1. This means we want to keep the deviation from the realised magnitude response close to 1 with a maximum error of $\Gamma{pb}$


$$ \mathbf{E}^{(pbe)}  = ||\mathbf{D}^{(pbe)} \boldsymbol{ \delta } +  \mathbf{d}^{(pbe)}  || $$

In [3]:
import gradients

In [4]:
w = np.arange(0, np.pi, 0.1)
x = dsp.add_tau(c, w)
D = gradients.H_mag_squared_gradient(x, w)

In [5]:
d = dsp.H_mag_squared(x, w)

In [6]:
delta = np.ones(D.shape[1])

In [7]:
np.matmul(D, delta) + d

array([1.10203845e+03, 1.15299105e+03, 5.38093884e+02, 2.54149460e+01,
       1.54357092e+00, 1.12860330e-01, 5.10898923e-03, 1.23550515e-04,
       1.15854744e-03, 1.28672502e-03, 9.18359094e-04, 5.07367553e-04,
       2.11645733e-04, 5.08516132e-05, 2.63134826e-07, 2.78457856e-05,
       1.06024823e-04, 2.14008887e-04, 3.37081718e-04, 4.65212526e-04,
       5.91755228e-04, 7.12421380e-04, 8.24530465e-04, 9.26476702e-04,
       1.01736261e-03, 1.09674262e-03, 1.16445136e-03, 1.22048777e-03,
       1.26493200e-03, 1.29789689e-03, 1.31949081e-03, 1.32979425e-03])

# Stopband constraints

In [8]:
gradients.H_gradient(x, w)

TypeError: jacrev requires real-valued outputs (output dtype that is a sub-dtype of np.floating), but got complex64. For holomorphic differentiation, pass holomorphic=True. For differentiation of non-holomorphic functions involving complex outputs, or function with integer outputs, use jax.vjp directly.